# Introduction

Notre approche pour la résolution de ce problème consiste en une série d’exécution de différents algorithmes vus en cours, soit :

- K plus proches voisins
- Arbre de decision
- Classification naïve bayésienne
- Perceptron
- Adaline
- Réseau de neurones
- Classification hiérarchique

Via une fonction permettant de déterminer la précision d’un modèle, en se basant sur un jeu de données d’entraînement et un jeu de données de test, nous procéderons en trois phases :

1. Implémentation des algorithmes avec leurs paramètres par défaut. Observation des performances. Cette phase sert à avoir une idée "de prime abord" de quels algorithmes seront à conserver par la suite.
2. Essais de ces mêmes algorithmes avec des paramètres différents. Sélection du jeu de paramètres le plus adéquat maximisant la précision de l’algorithme. Élimination des algorithmes les moins performants.
3. Création d’un ensemble de 10 instances du dataset fourni, avec un shuffling aléatoire différent. Création des vecteurs de données et de résultat, et exécution sur les algorithmes les plus précis filtrés en 2.

À l’issue de cette troisième phase, le meilleur algorithme sera celui qui obtiendra le meilleur score de précision.

> La plupart des algorithmes utilisés ici sont des classes provenant de la librairie Python Scikit-Learn. En cas d'usage d'une autre classe et, plus généralement, d'un code provenant d'une source tierce (internet, camarade), sa provenance sera systématiquement indiquée en commentaire.

# Dépendances

- python >= 3.11
- numpy >= 1.23.4
- pandas >= 1.5.2
- scikit-learn >= 1.1.3
- sklearn-hierarchical-classification >= 1.3.2

# Phase 1 : Essais avec l'ensemble des algorithmes non-paramétrés

## Importation du dataset, création des vecteurs X et y d'entraînement et de test

On récupère tout d'abord le jeu de données en .csv sous la forme d'un DataFrame, qu'on scinde avec un échantillon de 20% aléatoire pour les tests, et le reste pour l'entraînement.

In [175]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Source : https://datagy.io/pandas-shuffle-dataframe/#:~:text=One%20of%20the%20easiest%20ways,Dataframe%2C%20in%20a%20random%20order.
df = pd.read_csv("./diabetes.csv").sample(
    frac = 1,
    random_state=1
).reset_index()

y = df['Outcome']
X = df.drop('Outcome', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

## Création d'une fonction évaluant la précision de chaque algorithme

In [176]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier

from numpy import ndarray

def calculate_classifier_accuracy(
    X_train:ndarray,
    X_test:ndarray,
    y_train:ndarray,
    y_test:ndarray,
    classifier:KNeighborsClassifier|DecisionTreeClassifier|GaussianNB|Perceptron|MLPClassifier|KMeans|HierarchicalClassifier,
    return_confusion_matrix:bool=False
):
    classifier.fit(X_train,y_train)

    predicted = classifier.predict(X_test)

    accuracy = [True if predicted[i] == y_test[i] else False for i in range(len(predicted))]
    accuracy_stats = {
        "right": len([i for i in accuracy if i]),
        "wrong": len([i for i in accuracy if not i])
    }
    accuracy_stats["percentage"] = round((accuracy_stats["right"]/len(accuracy))*100,2)

    if return_confusion_matrix:
        accuracy_stats["confusion_matrix"] = {
        "true_positive": len([1 for i in range(len(predicted)) if (predicted[i] == y_test[i] and predicted[i] == 1)]),
        "true_negative": len([1 for i in range(len(predicted)) if (predicted[i] == y_test[i] and predicted[i] == 0)]),
        "false_positive":len([1 for i in range(len(predicted)) if (predicted[i] != y_test[i] and predicted[i] == 1)]),
        "false_negative":len([1 for i in range(len(predicted)) if (predicted[i] != y_test[i] and predicted[i] == 0)])
    }

    return accuracy_stats

## Évaluation des algorithmes

### K plus proches voisins

In [177]:
from sklearn.neighbors import KNeighborsClassifier

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, KNeighborsClassifier())

{'right': 102, 'wrong': 52, 'percentage': 66.23}

### Arbre de décision

In [178]:
from sklearn.tree import DecisionTreeClassifier

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, DecisionTreeClassifier())

{'right': 110, 'wrong': 44, 'percentage': 71.43}

### Classification naïve bayésienne

In [179]:
from sklearn.naive_bayes import GaussianNB

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, GaussianNB())

{'right': 111, 'wrong': 43, 'percentage': 72.08}

### Perceptron

In [180]:
from sklearn.linear_model import Perceptron

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, Perceptron())

{'right': 54, 'wrong': 100, 'percentage': 35.06}

### Adaline

Pour cet algorithme, n'ayant pas trouvé d'équivalent sur sklearn, nous utilisons le code de M. Ajitesh Kumar, trouvable sur le site VitalFlux.com, à l'adresse : https://vitalflux.com/adaline-explained-with-python-example/

In [181]:
import numpy as np

# Source : https://vitalflux.com/adaline-explained-with-python-example/
class CustomAdaline:
     
    def __init__(self, n_iterations=100, random_state=1, learning_rate=0.01):
        self.n_iterations = n_iterations
        self.random_state = random_state
        self.learning_rate = learning_rate
 
    '''
    Batch Gradient Descent
     
    1. Weights are updated considering all training examples.
    2. Learning of weights can continue for multiple iterations
    3. Learning rate needs to be defined
    '''
    def fit(self, X, y):
        rgen = np.random.RandomState(self.random_state)
        self.coef_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        for _ in range(self.n_iterations):
              activation_function_output = self.activation_function(self.net_input(X))
              errors = y - activation_function_output
              self.coef_[1:] = self.coef_[1:] + self.learning_rate*X.T.dot(errors)
              self.coef_[0] = self.coef_[0] + self.learning_rate*errors.sum()
     
    '''
    Net Input is sum of weighted input signals
    '''
    def net_input(self, X):
            weighted_sum = np.dot(X, self.coef_[1:]) + self.coef_[0]
            return weighted_sum
     
    '''
    Activation function is fed the net input. As the activation function is
    an identity function, the output from activation function is same as the
    input to the function.
    '''
    def activation_function(self, X):
            return X
     
    '''
    Prediction is made on the basis of output of activation function
    '''
    def predict(self, X):
        return np.where(self.activation_function(self.net_input(X)) >= 0.0, 1, 0)
     
    '''
    Model score is calculated based on comparison of
    expected value and predicted value
    '''
    def score(self, X, y):
        misclassified_data_count = 0
        for xi, target in zip(X, y):
            output = self.predict(xi)
            if(target != output):
                misclassified_data_count += 1
        total_data_count = len(X)
        self.score_ = (total_data_count - misclassified_data_count)/total_data_count
        return self.score_

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, CustomAdaline())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


{'right': 101, 'wrong': 53, 'percentage': 65.58}

### Réseau de neurones

In [182]:
from sklearn.neural_network import MLPClassifier

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, MLPClassifier())

{'right': 104, 'wrong': 50, 'percentage': 67.53}

### Classification hiérarchique

Cet algorithme de classification hiérarchique ne provient pas de Scikit-Learn à proprement parler, mais d'une librairie nommée `sklearn-hierarchical-classification`, proposant une classe de classification hiérarchique dérivée des algorithmes de Scikit-Learn. Cette librairie est disponible via pip et dispose d'un lien [PyPi](https://pypi.org/project/sklearn-hierarchical-classification/). Elle est maintenue par [Globality](https://www.globality.com/en-us).

In [183]:
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, HierarchicalClassifier())

{'right': 117, 'wrong': 37, 'percentage': 75.97}

On obtient donc les résultats suivants :

| Algorithme                      | Précision | Nombre de bonnes prédictions | Nombre de fausses prédictions |
|---------------------------------|-----------|------------------------------|-------------------------------|
| K plus proches voisins          | 66%       | 114                          | 52                            |
| Arbre de decision               | ≈72%*     | ≈110*                        | ≈44*                          |
| Classification naïve bayésienne | 72%       | 111                          | 43                            |
| Perceptron                      | 35%       | 54                           | 100                           |
| Adaline                         | 65,5%     | 101                          | 53                            |
| Réseau de neurones              | ≈61%*     | ≈103*                        | ≈51*                          |
| Classification hiérarchique     | 76%       | 117                          | 37                            |

<br>

> \* La génération de l'arbre de décision implique une certaine aléatoirité des résultats, cependant avec une précision toujours contenue entre 70.0 et 74.5.
<br>

> \* On observe une situation similaire avec le réseau de neurones, avec une précision oscillant entre 58.0 et 72.0.

# Phase 2 : Paramétrage des algorithmes et élimination

Notre objectif ici va être de rechercher, par itérations et essais successifs, les paramètres d'entrée de chaque algorithme donnant les meilleurs résultats, en terme de précision. Chaque algorithme se voit associé à un court paragraphe d'explication justifiant les choix de paramètres qui lui sont afférents.

### K plus proches voisins

Passer le nombre de plus proches voisins à considérer de 5 par défaut à 20 a permis d'augmenter le taux de précision vers ≈68%. L'augmentation de cette valeur au delà de 20 réduisait la précision, et en deçà la stabilisait à son niveau précédent.

Par défaut, chaque voisin est équipondéré lorsqu'on cherche à définir l'étiquette du nouveau point. En changeant la valeur de `weights`de `uniform` à `distance`, on utilise ainsi un système de pondération où plus le voisin est proche, plus il va être pondéré, permettant d'atteindre un taux de précision de ≈69.5%.

In [184]:
kn = KNeighborsClassifier(
    n_neighbors=20,
    weights="distance"
)

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, kn)

{'right': 107, 'wrong': 47, 'percentage': 69.48}

### Arbre de décision

La modification des paramètres de la classe, notamment du `criterion` (utilisant l'impureté de Gini par défaut) et du `splitter` (stratégie de division de chaque noeud, pouvant être "best" (la meilleure stratégie est déterminée par l'algorithme et appliquée) ou "random" (une stratégie de division de noeud aléatoire est appliquée)), n'ont pas permis d'améliorer les résultats. Toute tentative de paramétrage a mené l'algorithme a produire des résultats en deçà de 10 points ou plus de la moyenne de sa précision précédente.

La non-consistence des prédictions effecutées par cet algorithme (malgré un taux de bonnes prédictions élevé) ainsi que, par extension, la variabilité du résultat fourni, nous conforte dans le choix de son élimination.

In [185]:
dt = DecisionTreeClassifier()

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, dt)

{'right': 110, 'wrong': 44, 'percentage': 71.43}

### Classification naïve bayésienne

Cet algorithme présente deux paramètres :

- `priors` : Une liste de probabilités pour chaque classe. Cela implique de connaître ces dernières, celles-ci remplaçant celles calculées par l'algorithme. Cette valeur n'est pas définie par défaut.
- `var_smotthing` : Une valeur (inférieure à 1, par défaut à 1e-9) qui correspond à l'amplitude de la moyenne considérée dans la courbe de Gauss. Modifier cette valeur permet d'agrandir le champ adjacent à la moyenne et d'inclure plus de points dans la classification *([source](https://stackoverflow.com/questions/58046129/can-someone-give-a-good-math-stats-explanation-as-to-what-the-parameter-var-smoo))*.

Nous avons trouvé que toute valeur s'éloignant de la valeur par défaut de var_smoothing avait un impact nul, sinon négatif sur les performances de l'algorithme. Dans ce cas de figure, il s'agit également d'un algorithme pour lequel nous ne modifierons pas les paramètres dans une perspective d'amélioration de précision.

In [186]:
gnb = GaussianNB()

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, gnb)

{'right': 111, 'wrong': 43, 'percentage': 72.08}

### Perceptron

Il a été possible d'augmenter la précision du perceptron en jouant sur deux paramètres :
- `penalty` : Un type de régularisation permettant d'éviter une trop grande variabilité du modèle en terme de prédiction. Cette valeur peut être soit de "l1" (utilisant la [norme de manhattan](https://en.wikipedia.org/wiki/Taxicab_geometry)), "l2" (utilisant la norme euclidienne) ou "elasticnet" (une combinaison des deux précédentes). Par défaut, la pénalité est à None.
  L'utilité de ce paramètre peut être illustré par ces images d'exemple :

| Sans régularisation                                                                                | Avec régularisation                                                                                |
|----------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| <img src="https://github.com/christianversloot/machine-learning-articles/raw/main/images/poly_large.png" width="300px"> | <img src="https://github.com/christianversloot/machine-learning-articles/raw/main/images/poly_small.png" width="300px"> |

- `l1_ratio` : Lorsque `penalty` est défini sur "elasticnet", spécifie le pourcentage de l1 qui doit être appliqué par rapport à l2.


Nos tests ont montré que : 
- L'ajout d'une pénalité seule impliquait une baisse de environ 1% de la précision du modèle
- Cependant, l'ajout d'un l1_ratio a permis d'augmenter considérablement la précision du perceptron, lui octroyant près du double de sa précision d'origine. Toute valeur <= 0.4 n'a pas d'impact significatif. Le pic (64.94%) de précision est atteint à 0.5. Enfin, cette dernière décroît graduellement au fur et à mesure de l'augmentation du ratio.

La modification de ces paramètres ont permis de transformer le perceptron d'un algorithme peu intéressant à un algorithme offrant des performances honorables.

Les informations (ainsi que les images) que nous avons utilisé pour comprendre la régularisation proviennent d'un [article de M. Christian Versloot sur GitHub](https://github.com/christianversloot/machine-learning-articles/blob/main/what-are-l1-l2-and-elastic-net-regularization-in-neural-networks.md).

In [187]:
pcptr = Perceptron(
    penalty="elasticnet",
    l1_ratio=0.5
)

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, pcptr)

{'right': 100, 'wrong': 54, 'percentage': 64.94}

### Adaline

La modification d'aucun paramètre du code proposé par M. Ajitesh Kumar n'a permis d'impacter le résultat de l'algorithme, positivement ou négativement.

In [188]:
adl = CustomAdaline()

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, adl)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


{'right': 101, 'wrong': 53, 'percentage': 65.58}

### Réseau de neurones

La classe MLPClassifier de Scikit-Learn présente un grand nombre de paramètres. Parmi eux se trouvent notamment :
- `hidden_layer_size` : La taille de la couche cachée (par défaut à 100, les tests ont montré une absence d'amélioration en modifiant cette valeur)
- `activation` : La fonction d'activation de la couche cachée ("identity", "logistic", "tanh", "relu", par défaut à "relu"). Nos tests ont permis de déterminer que "logistic" permettait de diminuer la variabilité des résultats, et de conserver une précision haute par rapport aux extrêmes les plus bas précédents.
- `solver` : L'algorithme qui va déterminer la répartition des poids entre les couches du réseau. On a 3 possibilités :
  - "lbfgs" : fonctionne mieux sur des datasets larges, repose sur un principe de [méthodes quasi-Newton](https://fr.wikipedia.org/wiki/M%C3%A9thode_quasi-Newton).
  - "sgd" : Algorithme de descente de gradient standard
  - "adam" (par défaut) : Algorithme de descente de gradient amélioré
  L'utilisation d'un autre algorithme que "adam" n'a pas eu d'impact positif constaté sur les performances.
- `max_iter` : Le nombre maximum d'itérations de l'algorithme (par défaut à 200). Parfois, des warnings étaient affichés car le nombre maximum d'itérations était atteint avent que l'algorithme converge. Augmenter cette valeur n'a pas eu d'impact significatif sur la précision de l'algorithme, mais a permis d'éliminer ces warnings.

La classe `MLPClassifier` présente également d'autre paramètres afférents à, par exemple, certains solveurs spécifiques comme "adam" ou "sgd". Cependant, malgré l'exhaustivité des paramètres proposés, il ne nous apparaît pas pertinent d'utiliser un réseau de neurones pour cette tâche, au vu de la variabilité de sa précision.

In [189]:
mlpc = MLPClassifier(
    activation="logistic",
    max_iter=1000
)

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, mlpc)

{'right': 102, 'wrong': 52, 'percentage': 66.23}

### Classification hiérarchique

L'algorithme de classification hiérarchique de [Globality](https://pypi.org/project/sklearn-hierarchical-classification/) présente un ensemble de paramètres large. Cependant, aucun paramètre n'a permis d'augmenter la précision de l'algorithme.

Les paramètres que nous avons fait varier dans une optique d'amélioration de la précision sont :
- `base_estimator` : Un autre algorithme sklearn utilisé pour générer les sous-classifications. Par défaut, l'algorithme de base est déterminé automatiquement par la classe. Nous avons essayé avec l'ensemble des algorithmes vus dans ce notebook, avec une réduction de précision systématique alignée avec la précision de cet algorithme.
- `prediction_depth` : peut prendre les valeurs "mlnp" (mandatory leaf-node prediction) & "nmlnp" (non-mandatory leaf-node prediction, "nmlnp"). Détermine si l'algorithme doit s'arrêter après avoir calculé une feuille de l'arbre, ou si il peut s'arrêter avant à la condition que le taux de confiance de l'algorithme pour une prédiction soit inférieur à un seuil passé en paramètre.
- `stopping_criteria` : Un seuil (float) utile pour le paramètre `prediction_depth`
- `algorithm` : peut être "lcpn" (*local classifier per parent node*, par défaut) ou "lcn" (*local classifier per node*). Définit si un algorithme de classification doit être exécuté sur tous les noeuds parents, ou sur tous les noeuds dans l'absolu.
- `training_strategy` : Dans le cas d'utilisation de l'algorithme "lcn", définit la stratégie à appliquer ("exclusive", "less_exclusive", "inclusive", "less_inclusive", "siblings", "exclusive_siblings").

Il est à noter que la description des paramètres provient principalement des docstring de la classe `HierarchicalClassifier`. En effet, la documentation officielle du code [n'est plus disponible](https://code.globality.com/sklearn-hierarchical-classification/).

In [190]:
hc = HierarchicalClassifier()

calculate_classifier_accuracy(X_train, X_test, y_train, y_test, hc)

{'right': 117, 'wrong': 37, 'percentage': 75.97}

Ainsi, à l'issue des tests que nous avons effectués en modifiant les paramètres de chaque algorithme, nous pouvons dresser le tableau suivant :

| Algorithme                      | Précision | Évolution | Validation |
|---------------------------------|-----------|-----------|------------|
| K plus proches voisins          | 69,5%     | +3,5%     | ✅          |
| Arbre de decision               | ≈72%      | +0%       | ❌          |
| Classification naïve bayésienne | 72%       | +0%       | ✅          |
| Perceptron                      | 65%       | +30%      | ❌          |
| Adaline                         | 65.5%     | +0%       | ❌          |
| Réseau de neurones              | ≈66%      | +≈5%      | ❌          |
| Classification hiérarchique     | 76%       | +0%       | ✅          |

Nous avons choisi conserver trois algorithmes pour la troisième phase, qui consistera à analyser la consistence de leur précision sur différents échantillons entraînement / test.

Pour cette sélection, nous avons choisi :
- Les modèles qui n'ont pas un score variable, et
- Les modèles avec les trois meilleurs scores de précision

Ce qui nous amène à la conclusion que les algorithmes de classification les plus précis pour ce dataset sont :
1. Classification hiérarchique
2. Classification naïve bayésienne
3. K plus proches voisins

# Phase 3 : Analyse de la consistence des algorithmes

On commence ici par générer un ensemble de datasets dont le shuffling a été réalisé avec une seed aléatoire à chaque fois.

In [191]:
import pandas as pd

df = pd.read_csv("./diabetes.csv")

datasets_instances = [df.sample(frac=1, random_state=i).reset_index() for i in range(100)]

datasets_vectors = []

for dataset in datasets_instances:

    y = dataset['Outcome']
    X = dataset.drop('Outcome', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

    datasets_vectors.append({
        "X_train": X_train.to_numpy(),
        "X_test": X_test.to_numpy(),
        "y_train": y_train.to_numpy(),
        "y_test": y_test.to_numpy()
    })

classifiers_to_test = [
    KNeighborsClassifier(
        n_neighbors=20,
        weights="distance"
    ),
    GaussianNB(),
    HierarchicalClassifier()
]

test_results = []

for classifier in classifiers_to_test:
    scores = {
        "right": [],
        "wrong": [],
        "percentage": []
    }
    for dataset in datasets_vectors:
        result = calculate_classifier_accuracy(dataset["X_train"],dataset["X_test"],dataset["y_train"],dataset["y_test"],classifier)
        scores["right"].append(result["right"])
        scores["wrong"].append(result["wrong"])
        scores["percentage"].append(result["percentage"])
    
    test_results.append({
        "classifier": classifier,
        "average_right": sum(scores["right"])/len(scores["right"]),
        "average_wrong": sum(scores["wrong"])/len(scores["wrong"]),
        "average_percentage": round(sum(scores["percentage"])/len(scores["percentage"]),2)
    })

test_results

[{'classifier': KNeighborsClassifier(n_neighbors=20, weights='distance'),
  'average_right': 107.14,
  'average_wrong': 46.86,
  'average_percentage': 69.57},
 {'classifier': GaussianNB(),
  'average_right': 115.6,
  'average_wrong': 38.4,
  'average_percentage': 75.06},
 {'classifier': HierarchicalClassifier(),
  'average_right': 118.23,
  'average_wrong': 35.77,
  'average_percentage': 76.77}]

Ainsi, nous pouvons conclure que le meilleur algorithme de classification pour ce dataset est la ***classification hiérarchique***, avec dans le cas présent une précision moyenne de 76.77% sur 100 instances randomisées du dataset, contre 75.06% pour la classification naïve bayésienne et 69.67% pour l'algorithme des k plus proches voisins.